![OpenSARlab notebook banner](NotebookAddons/blackboard-banner.png)

# Subset Data Stack

### Alex Lewandowski; University of Alaska Fairbanks

This notebook crops a directory of tiffs to a subset area of interest using an interactive Matplotlib plot of an image in your data stack.

<img style="padding: 7px" src="NotebookAddons/UAFLogo_A_647.png" width="170" align="right"/></font>

**Important Note about JupyterHub**

Your JupyterHub server will automatically shutdown when left idle for more than 1 hour. Your notebooks will not be lost but you will have to restart their kernels and re-run them from the beginning. You will not be able to seamlessly continue running a partially run notebook.

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

## 0. Importing Relevant Python Packages

In this notebook we will use the following scientific library:

- [GDAL](https://www.gdal.org/) is a software library for reading and writing raster and vector geospatial data formats. It includes a collection of programs tailored for geospatial data processing. Most modern GIS systems (such as ArcGIS or QGIS) use GDAL in the background.

**Import the necesssary libraries and modules:**

In [ ]:
%%capture
from pathlib import Path
import json # for loads
import shutil

from osgeo import gdal
gdal.UseExceptions()
import pyproj 

from ipyfilechooser import FileChooser

from IPython.display import Markdown
from IPython.display import display
%matplotlib widget
    
import matplotlib.pyplot as plt 
plt.rcParams.update({'font.size': 12})

import opensarlab_lib as asfn
asfn.jupytertheme_matplotlib_format()

---
**Write functions to gather and print individual tiff paths:**

In [ ]:
def get_tiff_paths(paths):
    tiff_paths = list(paths.parent.rglob(paths.name))    
    tiff_paths.sort()
    return tiff_paths

def print_tiff_paths(tiff_paths):
    print("Tiff paths:")
    for p in tiff_paths:
        print(f"{p}\n")

**Select the directory holding your tiffs**
- Click the `Select` button
- Navigate to your data directory
- Click the `Select` button
- Confirm that the desired path appears in green text
- Click the `Change` button to alter your selection

In [ ]:
fc = FileChooser('/home/jovyan/notebooks')
display(fc)

**Determine the path to the analysis directory containing the tiff directory:**

In [ ]:
tiff_dir = Path(fc.selected_path)
analysis_dir = tiff_dir.parent
print(f"analysis_dir: {analysis_dir}")

paths = tiff_dir/"*.tif*"
tiff_paths = get_tiff_paths(paths)

**Determine the UTM zone for your images.** 

This assumes you have already reprojected multiple UTM projections to a single predominant projection using the Prepare_Data_Stack_Hyp3 notebook.

In [ ]:
info = gdal.Info(str(tiff_paths[0]), format='json')
info = info['coordinateSystem']['wkt']
utm = info.split('ID')[-1].split(',')[1][0:-2]
print(f"UTM Zone: {utm}")

In [ ]:
tiff_paths = get_tiff_paths(paths)
print_tiff_paths(tiff_paths)

**Create a string containing paths to one image for each area represented in the stack:**

In [ ]:
to_merge = {}
for pth in tiff_paths:
    info = (gdal.Info(str(pth), options = ['-json']))
    info = json.dumps(info)
    info = (json.loads(info))['wgs84Extent']['coordinates']
    
    coords = [info[0][0], info[0][3]]
    for i in range(0, 2):
        for j in range(0, 2):
            coords[i][j] = round(coords[i][j])
    str_coords = f"{str(coords[0])}{str(coords[1])}"
    if str_coords not in to_merge:
        to_merge.update({str_coords: pth})
merge_paths = ""
for pth in to_merge:
    merge_paths = f"{merge_paths} {to_merge[pth]}"
print(merge_paths)

**Merge the images for display in the Area-Of-Interest selector:**

In [ ]:
full_scene = analysis_dir/'full_scene.tif'

if full_scene.exists():
    full_scene.unlink()
    
gdal_command = f"gdal_merge.py -o {full_scene} {merge_paths}"
!{gdal_command}

---
## Subset The Tiffs

**Create a Virtual Raster Stack:**

In [ ]:
image_file = f"{analysis_dir}/raster_stack.vrt"
!gdalbuildvrt -separate $image_file -overwrite $full_scene

**Convert the VRT into an array:**

In [ ]:
img = gdal.Open(image_file)
rasterstack = img.ReadAsArray()

**Print the number of bands, pixels, and lines:**

In [ ]:
print(img.RasterCount) # Number of Bands
print(img.RasterXSize) # Number of Pixels
print(img.RasterYSize) # Number of Lines

**Create an AOI selector from an image in your raster stack:**

In [ ]:
fig_xsize = 7
fig_ysize = 7
aoi = asfn.AOI_Selector(rasterstack, fig_xsize, fig_ysize)

**Gather and define projection details:**

In [ ]:
geotrans = img.GetGeoTransform()
projlatlon = pyproj.Proj('EPSG:4326') # WGS84
projimg = pyproj.Proj(f'EPSG:{utm}')

**Write a function to convert the pixel, line coordinates from the AOI selector into geographic coordinates in the stack's EPSG projection:**

In [ ]:
def geolocation(x, y, geotrans,latlon=True):
    ref_x = geotrans[0]+x*geotrans[1]
    ref_y = geotrans[3]+y*geotrans[5]
    if latlon:
        ref_y, ref_x = pyproj.transform(projimg, projlatlon, ref_x, ref_y)
    return [ref_x, ref_y]

**Call geolocation to gather the aoi_coords:**

In [ ]:
try:
    aoi_coords = [geolocation(aoi.x1, aoi.y1, geotrans, latlon=False), geolocation(aoi.x2, aoi.y2, geotrans, latlon=False)]
    print(f"aoi_coords in EPSG {utm}: {aoi_coords}")
except TypeError:
    print('TypeError')
    display(Markdown(f'<text style=color:red>This error occurs if an AOI was not selected.</text>'))
    display(Markdown(f'<text style=color:red>Note that the square tool icon in the AOI selector menu is <b>NOT</b> the selection tool. It is the zoom tool.</text>'))
    display(Markdown(f'<text style=color:red>Read the tips above the AOI selector carefully.</text>'))

**Collect the paths to the tiffs:**

In [ ]:
tiff_paths = get_tiff_paths(paths)

**Create a subdirectory in which to store the subset tiffs:**

In [ ]:
print("Choose a directory name in which to store the subset geotiffs.")
print("Note: this will sit alongside the directory containing your pre-subset geotiffs.")
while True:
    sub_name = input()
    if sub_name == "":
        print("Please enter a valid directory name")
        continue
    else:
        break

**Subset the tiffs and move them from the individual product directories into their own directory, /tiffs:**

In [ ]:
tiff_paths = get_tiff_paths(paths)
for p in tiff_paths:
    print(p)

In [ ]:
subset_dir = analysis_dir/f'{sub_name}'

if not subset_dir.exists():
    subset_dir.mkdir()

# sometimes, tiff doesn't follow '[0-9]{7}T[0-9]6' format, hence just get the numbers in those cases 
for i, tiff_path in enumerate(tiff_paths):
    print(tiff_path)
    date = Path(asfn.date_from_product_name(str(tiff_path))).name.split('T')[0]
    polar = asfn.get_polarity_from_path(str(tiff_path))
    print(f"\nProduct #{i+1}:")
    gdal_command = f"gdal_translate -projwin {aoi_coords[0][0]} {aoi_coords[0][1]} {aoi_coords[1][0]} {aoi_coords[1][1]} -projwin_srs 'EPSG:{utm}' -co \"COMPRESS=DEFLATE\" -a_nodata 0 {tiff_path} {subset_dir}/{date}_{polar}.tiff"
    print(f"Calling the command: {gdal_command}")
    !{gdal_command}

**Delete any subset tifs that are filled with NaNs and contain no data.**

In [ ]:
subset_paths = subset_dir/f"*.tif*"
tiff_paths = get_tiff_paths(subset_paths)

asfn.remove_nan_filled_tifs(tiff_paths)

**Sometimes, when using gdal translate to subset a stack of images, there will be slight differences in sizes of the resulting images, off by a single pixel in either direction. The following code checks the newly subset stack for this problem, and if found, it re-subsets all the images to the size of the smallest image in the stack.**

In [ ]:
gdal_tiff_input = str(tiff_paths[0])

corners = [gdal.Info(gdal_tiff_input, format='json')['cornerCoordinates']['upperLeft'],
                 gdal.Info(gdal_tiff_input, format='json')['cornerCoordinates']['lowerRight']]

sizes = list()
for p in tiff_paths:
    info = gdal.Info(str(p), format='json')
    sizes.append((info['size'][0], info['size'][1]))
    corners = [[max(corners[0][0], gdal.Info(gdal_tiff_input, format='json')['cornerCoordinates']['upperLeft'][0]),
               min(corners[0][1], gdal.Info(gdal_tiff_input, format='json')['cornerCoordinates']['upperLeft'][1])],
               [min(corners[1][0], gdal.Info(gdal_tiff_input, format='json')['cornerCoordinates']['lowerRight'][0]),
               max(corners[1][1], gdal.Info(gdal_tiff_input, format='json')['cornerCoordinates']['lowerRight'][1])]]   
    
if len(sizes) != len(set(sizes)):
    print(set(sizes))
    print(corners)
    temp_path = analysis_dir/f"temp_subsetting"
    if not temp_path.exists():
        subset_dir.rename(temp_path)
        
        if not subset_dir.exists():
            subset_dir.mkdir()
            
        tiff_paths = list(temp_path.rglob('*.tif*'))

    for i, tiff_path in enumerate(tiff_paths):
        print(f"\nProduct #{i+1}:")
        gdal_command = f"gdal_translate -projwin {corners[0][0]} {corners[0][1]} {corners[1][0]} {corners[1][1]} -projwin_srs 'EPSG:{utm}' -co \"COMPRESS=DEFLATE\" -a_nodata 0 {tiff_path} {subset_dir}/{tiff_path.name}"
        print(f"Calling the command: {gdal_command}")
        !{gdal_command}
    shutil.rmtree(temp_path)

**Decide whether or not to cleanup the original tiffs:**

In [ ]:
cleanup = asfn.select_parameter(["Save original tiffs", "Delete original tiffs"], '')
cleanup

In [ ]:
if cleanup.value == 'Delete original tiffs':
    shutil.rmtree(tiff_dir)

**Print the path to your subset directory:**

In [ ]:
print(subset_dir)

*GEOS 657 Microwave Remote Sensing - Version 2.0.0 - November 2021*

*Version Changes:*
- *asf_notebook -> opensarlab-lib*
- *html -> markdown
- *url-widget*
- *%matplotlib widget*